In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_mldata
from IPython.display import display, clear_output 
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time
import csv
%matplotlib inline

### Import and Format Data

In [89]:
# import training.csv which contains a header row of face locations and 'image,' the column of lists of pixel values
with open("training.csv") as f:
    train_import = list(csv.reader(f))

# divide the rows 80% for training and 20% for development    
train_portion = int(.8*(len(train_import)))
dev_portion = len(train_import) - train_portion

# extract training and development data
train_data = train_import[1:train_portion]
dev_data = train_import[-dev_portion:]

# initialize the list of lists of pixel values 
train_image_data = []

# convert each list of pixel values (1 per row) to integers (first split the string up then convert each elem to int)
for row in range(len(train_data)):
    train_image_data.append([int(i) for i in list(train_data[row][30].split())])
    
# initialize array of the 30 different facial location target values (labels) for each example image
train_image_labels = np.empty((len(train_data), 30))

# iterate through each row and each example image's 30 target facial locations and covert to floats or NaNs
for i in range(len(train_data)):
    for j in range(len(train_data[i][0:30])):
        if train_data[i][j] == '':
            train_image_labels[i][j] = float('nan')
        else: 
            train_image_labels[i][j] = (float(train_data[i][j]))

### Linear Regression - Facial Recognition

In [92]:
tf.reset_default_graph()
trainX = tf.constant(train_image_data, dtype = tf.float32)
# only attempting 1 out of 30 facial locations (column 0)
trainY = tf.constant(train_image_labels[:][0], dtype = tf.float32)
learningRate = tf.constant(0.1)
# without initializer, variables are initializes with random values between -1 and 1.
beta = tf.get_variable('beta', shape=[9216])

def model(this_beta, x):
    return tf.multiply(this_beta, x)

def cost(this_beta):
    return tf.reduce_mean((model(this_beta, trainX) - trainY)**2) / 2

cc = cost(beta)

## Gradient descent by tensorflow
gd = tf.train.GradientDescentOptimizer(learningRate)
step = gd.minimize(cost(beta))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    cost_vec = []
    # By running step multiple times, we perform gradient descent for the variable beta. 
    # For each step, we can ouput the cost function.
    for i in range(150):
        _, cost = sess.run([step, cc])
        cost_vec.append(cost)
    plt.plot(cost_vec)
    my_beta = sess.run([beta])[0]
    print(my_beta)